In [1]:
import h5py
import numpy as np
from build_dataset import PoseVideo
import os
import moviepy.editor as mpy

In [2]:
def heatmap_to_bodypart_location(heatmap):
    return np.unravel_index(np.argmax(heatmap, axis=None), heatmap.shape)
    
def extract_central_frame(tensor):
    return tensor[0]

def predict_video(video):
    pass
    